In [1]:
from pyod.models.ecod import ECOD  # ECOD detector
from pyod.models.knn import KNN   # kNN detector
from joblib import dump, load
from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize
from loader import Loader
import os

In [2]:
clf_name = 'KNN'
# clf = ECOD()
clf = KNN()

In [4]:
loader = Loader()
if not os.path.exists('data/data.pkl'):
    df = loader.create_df_from_xlsx()
    loader.save_df(df)
else:
    df = loader.load_df()

100%|██████████| 449/449 [01:31<00:00,  4.91it/s]


In [5]:
print(df.shape)
print(df.columns)

(298263, 22)
Index(['کد محور', 'نام محور', 'زمان شروع', 'زمان پایان',
       'مدت زمان کارکرد (دقیقه)', 'تعداد کل وسیله نقلیه',
       'تعداد وسیله نقلیه کلاس 1', 'تعداد وسیله نقلیه کلاس 2',
       'تعداد وسیله نقلیه کلاس 3', 'تعداد وسیله نقلیه کلاس 4',
       'تعداد وسیله نقلیه کلاس 5', 'سرعت متوسط', 'تعداد تخلف سرعت غیر مجاز',
       'تعداد تخلف فاصله غیر مجاز', 'تعداد تخلف سبقت غیر مجاز',
       'تعداد برآورد شده', 'مدت زمان کارکرد(دقیقه)', 'تعداد وسیله نقلیه سبک',
       'تعداد وسیله نقلیه سنگین', 'تعداد تخلف سرعت غیرمجاز',
       'تعداد تخلف فاصله غیرمجاز', 'تعداد تخلف سبقت غیرمجاز'],
      dtype='object')


In [ ]:
clf.fit(X_train)

# get the prediction label and outlier scores of the training data
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

# get the prediction on the test data
y_test_pred = clf.predict(X_test)  # outlier labels (0 or 1)
y_test_scores = clf.decision_function(X_test)  # outlier scores

# it is possible to get the prediction confidence as well
y_test_pred, y_test_pred_confidence = clf.predict(X_test, return_confidence=True)  # outlier labels (0 or 1) and confidence in the range of [0,1]

In [ ]:
# evaluate and print the results
print("\nOn Training Data:")
evaluate_print(clf_name, y_train, y_train_scores)
print("\nOn Test Data:")
evaluate_print(clf_name, y_test, y_test_scores)

In [ ]:
visualize(clf_name, X_train, y_train, X_test, y_test, y_train_pred,
    y_test_pred, show_figure=True, save_figure=False)

In [ ]:
# save the model
dump(clf, 'clf.joblib')
# load the model
clf = load('clf.joblib')